# [手把手教你完成一个数据科学小项目（2）：数据提取、IP 查询](https://zhuanlan.zhihu.com/p/42151036)
本系列将全面涉及本项目从爬虫、数据提取与准备、数据异常发现与清洗、分析与可视化等细节，并将代码统一开源在GitHub：[DesertsX/gulius-projects](https://github.com/DesertsX/gulius-projects) ，感兴趣的朋友可以先行 star 哈。

请先阅读[“中国年轻人正带领国家走向危机”，这锅背是不背？](https://zhuanlan.zhihu.com/p/41880873)  一文，以对“手把手教你完成一个数据科学小项目”系列有个全局性的了解。上一篇文章[（1）数据爬取](https://zhuanlan.zhihu.com/p/42060094)里我讲解了如何用爬虫爬取新浪财经《中国年轻人正带领国家走向危机》一文的评论数据，其中涉及的抓包过程是蛮通用的，大家如果想爬取其他网站，也会是类似的流程，当然介绍的比较粗浅，可自行深入扩展学习。

本文将在上回爬到的数据基础上，进一步地完成数据的提取，同时调用（再写个爬虫）某 IP 查询网站由评论里的 IP 查询到更多信息）。

同样的，所有操作都是用 Python 的 pandas 等数据科学库完成的。

## 数据提取
### 读取数据

In [1]:
import pandas as pd
df = pd.read_csv('Sina_Finance_Comments_1_20180811.csv',encoding='utf-8')
df.head(10)

,page,jsons,cmntlist,replydict
0,1,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6B9A3E-75882626-13DE6E5B1-8C5-93D': [{'com...
1,2,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6D7DA2-B4A0B0AD-41AB47DC-8C5-825': [{'comm...
2,3,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6D2A44-B6CCA85E-50F84847-8C5-83E': [{'comm...
3,4,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6CE01A-DF68B2E4-177D4BA5D-8C5-8EA': [{'com...
4,5,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6C47FD-751FB7AF-462BF5DE-8C5-7F6': [{'comm...
5,6,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6BEA5B-597B393-ABE727AE-8C5-82F': [{'comme...
6,7,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6BE075-7EC43DAC-71F96BAA-8C5-91E': [{'comm...
7,8,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6BBE24-7BF5DB8B-749C0777-8C5-837': [{'comm...
8,9,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6BABD0-75540A2B-5911AE2D8DC7A94F-8C5-9AB':...
9,10,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6BA20B-27BB6899-6833BC13-8C5-8C8': [{'comm...


In [2]:
cmntlist = df.cmntlist.values.tolist()
print(len(cmntlist))
print(type(cmntlist[0])) # str 字符串格式
#print(cmntlist[0]) # 篇幅太长占地方
print(cmntlist[0][:200]) # 还是截取前200字符展示

191
<class 'str'>
[{'comment_imgs': '', 'parent_mid': '0', 'news_mid_source': '0', 'rank': '0', 'mid': '5B6B9FA6-777B83B3-68B55EBD-8C5-8E4', 'video': '', 'vote': '0', 'uid': '1756716733', 'area': '广东深圳', 'channel_sourc


In [6]:
replydict = df.replydict.values.tolist()
print(len(replydict))
print(type(replydict[0])) # str 字符串格式
#print(replydict[0]) # 篇幅太长占地方
print(replydict[0][:200])  # 还是截取前200字符展示

191
<class 'str'>
{'5B6B6C94-6A7ACA75-86B9ACEE-8C5-799': [{'comment_imgs': '', 'parent_mid': '0', 'news_mid_source': '0', 'rank': '0', 'mid': '5B6A4655-3B297465-E2D48224-8C5-7CD', 'video': '', 'vote': '0', 'uid': '3805


### eval() 一下
eval() 将 str 字符串格式转换成 list 或 dict 并由 apply 应用到这些列数据上

In [9]:
df['jsons'] = df.jsons.apply(lambda x: eval(x))  # eval() 将str字符串格式转换成 list 或 dict 并由 apply 应用到这些列数据上
df['cmntlist'] = df.cmntlist.apply(lambda x: eval(x))
df['replydict'] = df.replydict.apply(lambda x: eval(x))
df['hot_list'] = df.jsons.apply(lambda x: x['result']['hot_list']) # 补充提取下感兴趣的 hot_list 列数据，后续可能用到，待看
df.head()

,page,jsons,cmntlist,replydict,hot_list
0,1,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6B6C94-6A7ACA75-86B9ACEE-8C5-799': [{'comm...,"[{'comment_imgs': '', 'parent_mid': '0', 'news..."
1,2,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6C9525-6EA71538-17A3D8EEA-8C5-8D1': [{'com...,"[{'comment_imgs': '', 'parent_mid': '0', 'news..."
2,3,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6BEF87-75886535-93B54D73-8C5-880': [{'comm...,"[{'comment_imgs': '', 'parent_mid': '0', 'news..."
3,4,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6BE075-7EC43DAC-71F96BAA-8C5-91E': [{'comm...,"[{'comment_imgs': '', 'parent_mid': '0', 'news..."
4,5,"{'result': {'status': {'msg': '', 'code': 0}, ...","[{'comment_imgs': '', 'parent_mid': '0', 'news...",{'5B6BD03A-75884D11-716CEA51-8C5-91F': [{'comm...,"[{'comment_imgs': '', 'parent_mid': '0', 'news..."


查看 cmntlist 列的第一行的元素内容：

In [11]:
df.loc[0, 'cmntlist']

[{'against': '1533770764',
  'agree': '2',
  'area': '江苏南京',
  'audio': '',
  'channel': 'cj',
  'channel_source': '',
  'comment_imgs': '',
  'comment_mid': '0',
  'config': 'wb_verified=0&wb_screen_name=用户6525940293&wb_cmnt_type=comment_status&wb_user_id=6525940293&wb_description=&area=江苏南京&wb_parent=&wb_profile_img=http%3A%2F%2Ftvax2.sinaimg.cn%2Fdefault%2Fimages%2Fdefault_avatar_male_50.gif&wb_time=2018-08-09 07:26:02&wb_comment_id=4271006493624688',
  'content': '贷款买房你怎么说？[二哈][二哈]',
  'content_ext': '',
  'hot': '0',
  'ip': '49.90.86.218',
  'length': '17',
  'level': '0',
  'login_type': '0',
  'media_type': '0',
  'mid': '5B6B7C0A-315A56DA-184F9F245-8C5-89E',
  'news_mid': '0',
  'news_mid_source': '0',
  'newsid': 'comos-hhkuskt2879316',
  'newsid_source': '',
  'nick': '用户6525940293',
  'openid': '',
  'parent': '',
  'parent_mid': '0',
  'parent_nick': '',
  'parent_profile_img': '',
  'parent_uid': '0',
  'profile_img': 'http://tvax2.sinaimg.cn/default/images/default_avatar

## 准备工作
将 cmntlist 列的数据转换成列表格式，方便后面遍历和提取每条评论相关的数据

`cmntlists[0][0]` 为第一页第一个元素对应的评论数据，是字典形式，每条评论能拿到的数据就是这些，后面要提取的信息也主要是这些字段里筛选。

In [12]:
cmntlists = df['cmntlist'].values.tolist()
print(len(cmntlists),len(cmntlists[0]),cmntlists[0][0])# 分别为总页数，每页评论数，第一页第一个元素对应的评论数据

191 20 {'comment_imgs': '', 'parent_mid': '0', 'news_mid_source': '0', 'rank': '2', 'mid': '5B6B7C0A-315A56DA-184F9F245-8C5-89E', 'video': '', 'vote': '0', 'uid': '6525940293', 'area': '江苏南京', 'channel_source': '', 'content': '贷款买房你怎么说？[二哈][二哈]', 'nick': '用户6525940293', 'hot': '0', 'status_uid': '1663612603', 'content_ext': '', 'ip': '49.90.86.218', 'media_type': '0', 'config': 'wb_verified=0&wb_screen_name=用户6525940293&wb_cmnt_type=comment_status&wb_user_id=6525940293&wb_description=&area=江苏南京&wb_parent=&wb_profile_img=http%3A%2F%2Ftvax2.sinaimg.cn%2Fdefault%2Fimages%2Fdefault_avatar_male_50.gif&wb_time=2018-08-09 07:26:02&wb_comment_id=4271006493624688', 'channel': 'cj', 'comment_mid': '0', 'status': 'M_PASS', 'openid': '', 'newsid_source': '', 'parent': '', 'status_cmnt_mid': '4271006493624688', 'parent_profile_img': '', 'news_mid': '0', 'parent_nick': '', 'newsid': 'comos-hhkuskt2879316', 'parent_uid': '0', 'thread_mid': '0', 'thread': '', 'level': '0', 'against': '1533770764', 'us

知识点：sum() 可将列表嵌套列表的格式，整合到一个列表里，示例如下：

In [3]:
sum([[1,2],[3,2,1]], [])

[1, 2, 3, 2, 1]

cmntlists 原本是一个列表，里面每个元素是每页的20条评论组成的子列表，用 sum() 后就可以变成全部评论在一个 cmntlist 列表里的格式，方便后续遍历和提取数据

In [15]:
cmntlist = sum(cmntlists, [])
print(len(cmntlist)) # 总共的评论数据条数
cmntlist[0]  # 全部评论里的第一条，准确的说是最新一条的数据

3743


{'against': '1533770764',
 'agree': '2',
 'area': '江苏南京',
 'audio': '',
 'channel': 'cj',
 'channel_source': '',
 'comment_imgs': '',
 'comment_mid': '0',
 'config': 'wb_verified=0&wb_screen_name=用户6525940293&wb_cmnt_type=comment_status&wb_user_id=6525940293&wb_description=&area=江苏南京&wb_parent=&wb_profile_img=http%3A%2F%2Ftvax2.sinaimg.cn%2Fdefault%2Fimages%2Fdefault_avatar_male_50.gif&wb_time=2018-08-09 07:26:02&wb_comment_id=4271006493624688',
 'content': '贷款买房你怎么说？[二哈][二哈]',
 'content_ext': '',
 'hot': '0',
 'ip': '49.90.86.218',
 'length': '17',
 'level': '0',
 'login_type': '0',
 'media_type': '0',
 'mid': '5B6B7C0A-315A56DA-184F9F245-8C5-89E',
 'news_mid': '0',
 'news_mid_source': '0',
 'newsid': 'comos-hhkuskt2879316',
 'newsid_source': '',
 'nick': '用户6525940293',
 'openid': '',
 'parent': '',
 'parent_mid': '0',
 'parent_nick': '',
 'parent_profile_img': '',
 'parent_uid': '0',
 'profile_img': 'http://tvax2.sinaimg.cn/default/images/default_avatar_male_50.gif',
 'rank': '2',
 

遍历下拿到 ip 和城市数据。因为知道有 ip 查询的网站，所以本回也想由此看看能否拿到更多的数据。此外城市数据结合评论时间也可以绘制全国热力图等，可见此前的文章：[（送福利）BDP绘制微博转发动态热力图](https://zhuanlan.zhihu.com/p/29557747)、[酷炫的地图可视化作品（一）](https://zhuanlan.zhihu.com/p/30763121)，效果如下，看起来很复杂，其实实现过程很简单，拖曳下数据就行，无需编程，怎么做的可以看第一个链接。看图里的时间也正好是一年前了，时光荏苒.....
<img src='images/weibo-repo-map-by-BDP.gif'>

In [16]:
for num,cmnt in enumerate(cmntlist):
    print(cmnt['ip'], cmnt['area'])
    if num==10:break

49.90.86.218 江苏南京
59.44.235.75 辽宁铁岭
110.53.17.193 湖南怀化
117.136.32.92 广东广州
49.66.22.166 江苏无锡
117.136.0.149 北京
106.122.202.117 福建厦门
106.122.202.117 福建厦门
221.220.138.231 北京
39.68.204.68 山东济宁
219.232.34.181 北京


网上搜下 ip 查询的网站，本回用的是：https://ip.cn/ 想换成其他网站的朋友也可以自行写爬虫实现。 

ip 查询的原理并不了解，拿到的数据是否准确也无从考证，本回也仅是根据此信息来挖掘，不过和原本评论数据里自带的城市可以对照下，发现大多是一致的，数据应该还算靠谱。 

另外注意到科学上网后图中显示的 ip 就在美国了，后面查询评论里的 ip 时也有很多一些海外的，不知道是在当地呢，还是在国内但也是科学上网的，无从知晓。
<img src='images/ip-cn.jpg'>

In [18]:
import requests
def ip2loc(ip):
    url = 'https://ip.cn/index.php?ip={}'.format(ip)
    r = requests.get(url).text
    return r
text = ip2loc(cmntlist[0]['ip'])
print(text)

<!DOCTYPE html>
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<link href="//s.ip-cdn.com" rel="dns-prefetch" />
<title>49.90.86.218 - IP.cn - IP 地址查询 | 地理位置 | 手机归属地</title>
<meta name="robots" content="all" />
<meta name="Keywords" content="ip,ip查询,手机ip,本机ip,外网ip,ip地址查询,手机号,归属地,49.90.86.218">
<meta name="Description" content="专业本机 IP 地址查询、手机 IP 地址、地理位置查询、IP 数据库、手机号归属地查询、电话号码黄页查询，可查广告、骚扰、快递、银行、保险、房地产、中介电话。">
<link href='//s.ip-cdn.com/css/bootstrap.min.css' rel='stylesheet' type='text/css'>
<meta name="viewport" content="width=device-width, minimum-scale=0.5">
<meta name="format-detection" content="telephone=no">
<!--[if lt IE 8]>
<script src="//s.ip-cdn.com/js/ie8.js"></script>
<![endif]-->
<link href='//s.ip-cdn.com/css/main.css' rel='stylesheet' type='text/css'>
</head>
<body onLoad="document.fs.ip.focus()">
<div class="container-fluid">
	<div class="header">
		<a href="/"><img src="//s.ip-cdn.com/img/logo.gif"></a>
	</div>

	<div class="mainbar">


ip 查询返回的结果主要在这里，用正则表达式 re / BeautifulSoup /lxml xpath 等进行相应的提取都行：
> `<div id="result"><div class="well"><p>您查询的 IP：<code>117.170.240.254</code></p>`
`<p>所在地理位置：<code>江西省萍乡市 移动</code></p>`
`<p>GeoIP: Pingxiang, Jiangxi, China</p><p>China Mobile Guangdong</p>`  
 
[Python爬虫利器三之Xpath语法与lxml库的用法](https://cuiqingcai.com/2621.html)   
xpath 路径可以在网页上选中需要提取的元素，然后右键“审查元素”，然后就能 copy xpath 路径了

In [21]:
from lxml import etree
html = etree.HTML(text)
loc = html.xpath('//div[@id="result"]/div/p[2]/code/text()')[0]
tele = html.xpath('//div[@id="result"]/div/p[4]/text()')[0]
geo_ip = html.xpath('//div[@id="result"]/div/p[3]/text()')[0]
print(loc,'*',tele,'*',geo_ip)

江苏省南京市 电信 * China Telecom * GeoIP: Nanjing, Jiangsu, China


## IP 查询测试一下

In [24]:
%%time 
# 耗时 Wall time: 36 s
import requests
from lxml import etree
import time
import random
def ip2loc(ip):
    url = 'https://ip.cn/index.php?ip={}'.format(ip)
    text = requests.get(url).text
    html = etree.HTML(text)
    try:
        loc = html.xpath('//div[@id="result"]/div/p[2]/code/text()')[0]
    except:loc=''
    try:
        geo_ip = html.xpath('//div[@id="result"]/div/p[3]/text()')[0]
    except:geo_ip=''
    try:
        tele = html.xpath('//div[@id="result"]/div/p[4]/text()')[0]
    except:tele=''
    return loc+' * '+geo_ip+' * '+tele

for num,cmnt in enumerate(cmntlist):
    ip_loc = ip2loc(cmnt['ip'])
    print(num, cmnt['ip'], cmnt['area'], ip_loc)
    if num%5==0:
        time.sleep(random.randint(0,2))
    if num==30:break

0 49.90.86.218 江苏南京 江苏省南京市 电信 * GeoIP: Nanjing, Jiangsu, China * China Telecom
1 59.44.235.75 辽宁铁岭 辽宁省铁岭市 电信 * GeoIP: Changchun, Jilin, China * China Telecom Liaoning
2 110.53.17.193 湖南怀化 湖南省怀化市 联通 * GeoIP: Huaihua, Hunan, China * China Unicom HuNan
3 117.136.32.92 广东广州 广东省东莞市 移动 * GeoIP: Guangzhou, Guangdong, China * China Mobile
4 49.66.22.166 江苏无锡 江苏省无锡市 电信 * GeoIP: Wuxi, Jiangsu, China * China Telecom jiangsu
5 117.136.0.149 北京 北京市 移动 * GeoIP: Beijing, China * China Mobile
6 106.122.202.117 福建厦门 福建省厦门市 电信 * GeoIP: Xiamen, Fujian, China * China Telecom
7 106.122.202.117 福建厦门 福建省厦门市 电信 * GeoIP: Xiamen, Fujian, China * China Telecom
8 221.220.138.231 北京 北京市 联通 * GeoIP: Beijing, China * China Unicom Beijing
9 39.68.204.68 山东济宁 山东省济宁市 联通 * GeoIP: Jinan, Shandong, China * China Unicom Shandong
10 219.232.34.181 北京 北京市 方正宽带 * GeoIP: Beijing, China * China Telecom Beijing
11 130.88.240.72 英国英格兰 英国  * GeoIP: Manchester, United Kingdom * The University of Manchester
12 182.149.207.209 四川成都 四

`{'nick': '用户6525940293', 'time': '2018-08-09 07:26:04', 'content': '贷款买房你怎么说？[二哈][二哈]', 'area': '江苏南京',  'ip': '49.90.86.218',  'length': '17', 'against': '1533770764',  'agree': '2',  'channel': 'cj',  'hot': '0',  'level': '0',  'login_type': '0',  'media_type': '0',  'mid': '5B6B7C0A-315A56DA-184F9F245-8C5-89E',

'audio': '', 'channel_source': '',  'comment_imgs': '', 'comment_mid': '0',
 'config': 'wb_verified=0&wb_screen_name=用户6525940293&wb_cmnt_type=comment_status&wb_user_id=6525940293&wb_description=&area=江苏南京&wb_parent=&wb_profile_img=http%3A%2F%2Ftvax2.sinaimg.cn%2Fdefault%2Fimages%2Fdefault_avatar_male_50.gif&wb_time=2018-08-09 07:26:02&wb_comment_id=4271006493624688',
 'content_ext': '',   'news_mid': '0',  'news_mid_source': '0',  'newsid': 'comos-hhkuskt2879316',  'newsid_source': '', 
 'openid': '',  'parent': '',  'parent_mid': '0',  'parent_nick': '',  'parent_profile_img': '',  'parent_uid': '0',
 'profile_img': 'http://tvax2.sinaimg.cn/default/images/default_avatar_male_50.gif',
 'rank': '2',  'status': 'M_PASS',  'status_cmnt_mid': '4271006493624688',  'status_uid': '1663612603',  'thread': '',  'thread_mid': '0',
  'uid': '6525940293',  'usertype': 'wb',  'video': '',  'vote': '0'}`

## 数据提取与保存
需查询3千多个 IP ，所以这一步耗些时间，设置间隔时间，以免对 IP 查询的网站造成侵扰（可以已经造成了，逃...也希望看到并运行这部分代码的朋友可以注意下，设置间隔时间久些，或换个其他查询网站，就怕看到的人多，都去访问该网站，没准要出幺蛾子，瑟瑟发抖）。

In [26]:
%%time
import time
import random
# 
sinanews_comments = pd.DataFrame(columns = ['No','page','nick','time','content','area',
                                            'ip','ip_loc','length','against','agree', 'channel', 
                                            'hot', 'level', 'login_type', 'media_type', 'mid'])
page=1
for num,cmnt in enumerate(cmntlist):
    nick = cmnt['nick'] 
    times = cmnt['time'] #命名成 time 会和下面 time.sleep() 冲突 # 所以命名成 times
    content = cmnt['content']
    area = cmnt['area']
    ip = cmnt['ip']
    ip_loc = ip2loc(cmnt['ip'])
    length = cmnt['length']
    against = cmnt['against']
    agree = cmnt['agree']
    channel = cmnt['channel']
    hot = cmnt['hot']
    level = cmnt['level']
    login_type = cmnt['login_type']
    media_type = cmnt['media_type']
    mid = cmnt['mid']
    print(num+1,page,times,nick,content,area,ip_loc)        
    sinanews_comments = sinanews_comments.append({'No':num+1,'page':page,'nick':nick,'time':times,'content':content,'area':area,
                                                  'ip':ip,'ip_loc':ip_loc,'length':length,'against':against,'agree':agree,
                                                  'channel':channel,'hot':hot,'level':level,'login_type':login_type,
                                                  'media_type':media_type,'mid':mid},ignore_index=True)
    if num%30 == 0:
        time.sleep(random.randint(0,1))
    if int((num+1)%20) == 0:
        page += 1
sinanews_comments.to_csv('Sina_Finance_Comments_All.csv', encoding='utf-8', line_terminator='\r\n')

1 1 2018-08-09 07:26:04 用户6525940293 贷款买房你怎么说？[二哈][二哈] 江苏南京 江苏省南京市 电信 * GeoIP: Nanjing, Jiangsu, China * China Telecom
2 1 2018-08-09 07:23:40 为什么要开网店 我们在高额贷款利息下 能活着就不错了 辽宁铁岭 辽宁省铁岭市 电信 * GeoIP: Changchun, Jilin, China * China Telecom Liaoning
3 1 2018-08-09 07:22:41 了了了了丶 有链接吗[doge] 湖南怀化 湖南省怀化市 联通 * GeoIP: Huaihua, Hunan, China * China Unicom HuNan
4 1 2018-08-09 07:12:42 张儁乂 你家庭杠杆加到几多？ 广东广州 广东省东莞市 移动 * GeoIP: Guangzhou, Guangdong, China * China Mobile
5 1 2018-08-09 06:54:49 用户6618549650 胡说八道 江苏无锡 江苏省无锡市 电信 * GeoIP: Wuxi, Jiangsu, China * China Telecom jiangsu
6 1 2018-08-09 06:21:10 不一般的杜 别操心了，早睡早起吧 北京 北京市 移动 * GeoIP: Beijing, China * China Mobile
7 1 2018-08-09 06:20:04 百利今天胖了吗 这里说的是借钱啊 福建厦门 福建省厦门市 电信 * GeoIP: Xiamen, Fujian, China * China Telecom
8 1 2018-08-09 06:18:30 百利今天胖了吗 这里说的是借钱消费啊，自己赚的当然无所谓 福建厦门 福建省厦门市 电信 * GeoIP: Xiamen, Fujian, China * China Telecom
9 1 2018-08-09 06:16:39 icecream1013 人在家中躺，锅从天上来😂 北京 北京市 联通 * GeoIP: Beijing, China * China Unicom Beijing
10 1 2018-08-09 0

69 4 2018-08-09 14:34:30 不会笑的柴郡猫 开心就好啊，摸摸 日本 日本  * GeoIP: Japan * Softbank BB
70 4 2018-08-09 14:30:12 木木徽 媒体还要点脸吗？怎么不看看根本原因呢？ 安徽合肥 安徽省合肥市 移动 * GeoIP: China * China Mobile Guangdong
71 4 2018-08-09 14:29:04 翔鹤-瑞鹤-赤城-加贺-苍龙-飞龙 叫你们工作是用来赚钱上缴国家的，你们竟然去买星巴克提高生活水平，是可忍孰不可忍 法国 法国  * GeoIP: Paris, Ile-de-France, France * SFR
72 4 2018-08-09 14:18:41 巴吧嘟啦 为经济危机背锅！系统性金融风险可不能怪消费，这是政策和贸易的问题 广东佛山 广东省佛山市 视通宽带 * GeoIP: Jiangmen, Guangdong, China * Guangdong Cable Corporation Limited
73 4 2018-08-09 14:16:36 小生有礼_ 国家不是正想拉动内需吗? 广东东莞 广东省东莞市 电信 * GeoIP: Shenzhen, Guangdong, China * China Telecom Guangdong
74 4 2018-08-09 14:15:42 小灬熊熊 的是命，哈哈 重庆 重庆市 移动 * GeoIP: Chongqing, China * China Mobile Guangdong
75 4 2018-08-09 14:15:13 用户6394255872 不婚主义，不生孩子，不买房（根本买不起）， 上海 上海市 鹏博士 * GeoIP: Beijing, China * BeiJing Guoxin bilin Telecom Technology Co.,Ltd
76 4 2018-08-09 14:15:02 YMM客源_g47  哇塞，真不错 河南漯河 河南省漯河市 联通 * GeoIP: Zhengzhou, Henan, China * China Unicom Henan
77 4 2018-08-09 14:13:53 浮游道人 [允悲]早点休息 美帝人民为我国操什么心… 北京

134 7 2018-08-09 10:24:46 gnuiphuejz 谁在带领中国心里没数吗[doge] 上海 上海市 移动 * GeoIP: Shanghai, China * Shanghai Mobile Communications Co.,Ltd.
135 7 2018-08-09 10:19:01 醉生199211 小编应该有个正在叛逆期的孩子，大家理解一下 北京 北京市 电信 * GeoIP: Beijing, China * ChinaTelecom Group Beijing Ltd,Co
136 7 2018-08-09 10:18:27 用户6532907466 年轻人的钱都被讹完了，当然只能找你们借文革时砸庙夺宝挣的钱咯 香港 香港特别行政区 中国移动 * GeoIP: Central District, Hong Kong * China Mobile
137 7 2018-08-09 10:17:49 想换名的橘子 要是啥都不买，攒钱买房，不说买不起泡沫了呢？攒钱炒股P2P还不如花了呢 北京 北京市 电信 * GeoIP: Beijing, China * China Telecom Beijing
138 7 2018-08-09 10:16:02 Tony小天天 那银行也别开了，全是看借贷养活的。 四川成都 四川省成都市 移动 * GeoIP: Chengdu, Sichuan, China * China Mobile Guangdong
139 7 2018-08-09 10:15:26 大大大大大不太大 ？？？？？？来晚了，请问是直接笑吗？ 广东东莞 广东省东莞市 电信 * GeoIP: Guangzhou, Guangdong, China * China Telecom Guangdong
140 7 2018-08-09 10:14:09 令狐up 主要没钱 江西九江 江西省九江市 移动 * GeoIP: Jiujiang, Jiangxi, China * China Mobile Guangdong
141 8 2018-08-09 10:11:38 杨某人F 杨老师，爱上了，说的太对了。 江西九江 江西省九江市 移动 * GeoIP: Ruichang, Jiangxi, China * China Mobile

197 10 2018-08-09 07:40:02 吃芒果小能手么么哒 ojbk 北京 北京市 联通 * GeoIP: Beijing, China * China Unicom Beijing
198 10 2018-08-09 07:39:22 海边游子_57581 狗屁 江苏苏州 江苏省苏州市 电信 * GeoIP: Shanghai, China * China Telecom jiangsu
199 10 2018-08-09 07:37:24 叫我白木好了 还想让我贷款买房噢 云南曲靖 云南省昆明市 联通 * GeoIP: Kunming, Yunnan, China * China Unicom Yunnan
200 10 2018-08-09 07:27:44 白草堂 年轻人不背锅 河南驻马店 河南省驻马店市 移动 * GeoIP: Zhumadian, Henan, China * Henan Mobile Communications Co.,Ltd
201 11 2018-08-09 07:26:04 用户6525940293 贷款买房你怎么说？[二哈][二哈] 江苏南京 江苏省南京市 电信 * GeoIP: Nanjing, Jiangsu, China * China Telecom
202 11 2018-08-09 07:23:40 为什么要开网店 我们在高额贷款利息下 能活着就不错了 辽宁铁岭 辽宁省铁岭市 电信 * GeoIP: Changchun, Jilin, China * China Telecom Liaoning
203 11 2018-08-09 07:22:41 了了了了丶 有链接吗[doge] 湖南怀化 湖南省怀化市 联通 * GeoIP: Huaihua, Hunan, China * China Unicom HuNan
204 11 2018-08-09 07:12:42 张儁乂 你家庭杠杆加到几多？ 广东广州 广东省东莞市 移动 * GeoIP: Guangzhou, Guangdong, China * China Mobile
205 11 2018-08-09 06:54:49 用户6618549650 胡说八道 江苏无锡 江苏省无锡市 电信 * GeoIP: Wuxi, Jiangsu, Chin

264 14 2018-08-08 23:43:03 TroubleIm ？？？ 上海徐汇 上海市 鹏博士 * GeoIP: Shanghai, China * Shanghai Great Wall Broadband Network Service Co.
265 14 2018-08-08 23:35:30 fossildim 笑死了 江苏南京 江苏省南京市 电信 * GeoIP: Nanjing, Jiangsu, China * China Telecom jiangsu
266 14 2018-08-08 23:26:03 安普露丝亚 首先工程师没你想的那么老，其次关工程师屁事？ 北京 北京市 鹏博士 * GeoIP: Beijing, China * Beijing Teletron Telecom Engineering Co.
267 14 2018-08-08 23:25:42 他的心情似乎不太好 其实这篇文章和国家也没关系啊，是外国媒体报道的，标题也是他们起的 广西玉林 广西自治区玉林市 移动 * GeoIP: Yulin, Guangxi, China * China Mobile Guangdong
268 14 2018-08-08 23:25:32 尤物少女kiko 我一天到晚，手机十几条信息，中信，浦发，建行，各大银行给我发信息要我开信用卡。现在物价这么高，工资够吃住，还攒钱，攒个十年八年还不够你一个星期楼价涨得快，活在当下就是这么来的，既然都买不起，那还不如旅游喝咖啡[二哈] 广东广州 广东省广州市 联通 * GeoIP: Guangzhou, Guangdong, China * China Unicom Guangzhou network
269 14 2018-08-08 23:25:30 囧囧小宅女 如果不买房又不生孩子，过得不要太潇洒 江苏苏州 江苏省苏州市 电信 * GeoIP: Suzhou, Jiangsu, China * China Telecom
270 14 2018-08-08 23:23:50 脑细菌_GIVE_ME_A_DRAGON [思考]好有道理 甘肃兰州 甘肃省 移动 * GeoIP: Lanzhou, Gansu, China * China Mobile Guangdong
271 14 20

...

3508 176 2018-08-07 23:05:43 GUO哈哈哈哈哈 明明是你们带领的，咋能怪90后了。他们都还没机会带领任何人呢 湖南 湖南省 移动 * GeoIP: Changsha, Hunan, China * China Mobile
3509 176 2018-08-07 23:05:37 狄羿文 我代表90后向国家认错，对不起…… 吉林长春 吉林省长春市 联通 * GeoIP: Changchun, Jilin, China * China Unicom Jilin
3510 176 2018-08-07 23:05:33 郑伟月月鸟 首先，现在九零后混成什么样子跟社会环境是分不开的，然后……外出走路小心香蕉皮～香蕉皮不一定是九零后扔～ 广东中山 广东省中山市 电信 * GeoIP: Zhongshan, Guangdong, China * China Telecom Guangdong
3511 176 2018-08-07 23:05:32 朱哥爱咖啡 是谁带领国家走向危机。大家的心里明镜似的。 日本岐阜县 日本  * GeoIP: Nagakute, Aichi, Japan * Softbank BB
3512 176 2018-08-07 23:04:55 走向大清 国足 广东深圳 广东省深圳市 联通 * GeoIP: Shenzhen, Guangdong, China * China Unicom Shenzen network
3513 176 2018-08-07 23:04:43 看世界社会经济见闻 这都能想出来。中国人几千年的攀比心理是绝对不会变的，而只要还有攀比文化，中国在经济发展上就一定领跑世界。当然前提是我们自己稳 广东佛山 广东省广州市 联通 * GeoIP: Guangzhou, Guangdong, China * China Unicom Guangzhou network
3514 176 2018-08-07 23:04:33 克罗诺皮奥与法玛的故事 @我就是那个Super 脑子有屎吧？ 重庆 重庆市 移动 * GeoIP: Chongqing, China * China Mobile Guangdong
3515 176 2018-08-07 23:04:10 凳凳微视频 这么大一个锅，这他么到哪里说

3572 179 2018-08-07 22:32:42 雪地霜狼 这就是有一个从供给方向来解释需求方向的言论，这些人在写财经报道的时候不知道有没有好好认真读一下经济学最基本的供给需求原理 北京 北京市 联通 * GeoIP: Beijing, China * China Unicom Beijing
3573 179 2018-08-07 22:32:35 走路走中间 我原本生活过得还行，自从当上房奴我特么买根裤子都要考虑再三 四川成都 四川省成都市 电信 * GeoIP: Chengdu, Sichuan, China * China Telecom Sichuan
3574 179 2018-08-07 22:32:27 木沐雨点 咋滴，打我啊[哈哈][哈哈][污] 广东广州 广东省东莞市 移动 * GeoIP: Guangzhou, Guangdong, China * China Mobile
3575 179 2018-08-07 22:32:25 Caaaaaasss 评论优秀系列 美国 美国  * GeoIP: Denver, Colorado, United States * T-Mobile USA
3576 179 2018-08-07 22:31:34 微小思想者 然而毕竟买不起房。 浙江杭州 浙江省杭州市 电信 * GeoIP: Hangzhou, Zhejiang, China * China Telecom Hangzhou
3577 179 2018-08-07 22:30:09 爱生活也爱扣扣 锅从天上来 江苏南京 江苏省南京市 东南大学 * GeoIP: Nanjing, Jiangsu, China * Southeast University
3578 179 2018-08-07 22:29:58 白居居白 喝星巴克重点是拍照发朋友圈 广东深圳 广东省深圳市 移动 * GeoIP: Shenzhen, Guangdong, China * China Mobile
3579 179 2018-08-07 22:29:50 怪师父MAX 不需要为房子操心吧，父母、爷爷奶奶、外公外婆的房子，将来不都是他们的？ 浙江湖州 浙江省湖州市 电信 * GeoIP: Beijing, China * China Telecom Huzho

3637 182 2018-08-07 22:07:44 Mr亚古兽 人在家中坐，祸从天上来 山东济南 山东省 联通 * GeoIP: Jinan, Shandong, China * China Unicom Shandong
3638 182 2018-08-07 22:07:38 守望日晞 这锅甩的厉害 江西南昌 江西省南昌市 联通 * GeoIP: Nanchang, Jiangxi, China * China Unicom Jiangxi
3639 182 2018-08-07 22:06:26 雨天遇见泥 我特么把家里买给我结婚的新房子让我父母改善了居住条件，自己和媳妇硬是贷了80万买了房的年轻人…这锅看来我是只能背了[允悲] 山西大同 山西省大同市 联通 * GeoIP: Datong, Shanxi, China * Sxdt-datong-BAS
3640 182 2018-08-07 22:06:22 小李書樹 我看了好气！ 上海 上海市 联通 * GeoIP: Shanghai, China * China Unicom Shanghai network
3641 183 2018-08-07 22:06:20 dany大叔 这锅太黑，没法背 广东深圳 广东省深圳市 联通 * GeoIP: Shenzhen, Guangdong, China * China Unicom Shenzen network
3642 183 2018-08-07 22:05:28 我心中尚未崩坏的地方2018 人在家中坐，锅从天上来[摊手][摊手] 河北石家庄 河北省石家庄市 联通 * GeoIP: Shijiazhuang, Hebei, China * China Unicom Hebei
3643 183 2018-08-07 22:03:49 北约之盟 从评论来看，确实如此 江苏苏州 江苏省苏州市 电信 * GeoIP: Nanjing, Jiangsu, China * China Telecom jiangsu
3644 183 2018-08-07 22:03:02 好多白果_cgc 老人永远不会错[并不简单] 山东烟台 山东省烟台市 联通 * GeoIP: Yantai, Shandong, China * China Unicom Liaoni

3702 186 2018-08-07 21:47:00 社会主义接头人 还有什么锅尽管来，年轻人身体好不怕，我们继续负重前行[doge][doge][doge][doge][doge][doge] 内蒙古呼和浩特 内蒙古自治区通辽市 联通 * GeoIP: Hohhot, Nei Mongol, China * China Unicom Liaoning
3703 186 2018-08-07 21:46:54 巴马心态 真心带不动！！！ 北京 北京市 联通 * GeoIP: Beijing, China * China Unicom Beijing
3704 186 2018-08-07 21:46:47 热心市民金小姐 这锅甩的牛逼 贵州贵阳 贵州省贵阳市 电信 * GeoIP: Guiyang, Guizhou, China * China Telecom Guizhou
3705 186 2018-08-07 21:45:45 EyreFree 抱歉，穷困潦倒的年轻人不背这锅，[吃瓜] 上海 上海市 电信 * GeoIP: Shanghai, China * China Telecom Shanghai
3706 186 2018-08-07 21:45:41 怪Doolittle 。。。轮到00后背锅了 北京 北京市 移动 * GeoIP: Beijing, China * China Mobile
3707 186 2018-08-07 21:45:34 Isaiah_9891 年轻人无罪。 山西太原 山西省太原市 电信 * GeoIP: Taiyuan, Shanxi, China * China Telecom Shanxi
3708 186 2018-08-07 21:45:27 Zzz-connect 作为大西北嘉峪关的孩子这锅我不背 甘肃兰州 甘肃省 移动 * GeoIP: Lanzhou, Gansu, China * China Mobile Guangdong
3709 186 2018-08-07 21:45:04 Adam_Rong 这锅是年轻人的吗？ 江苏苏州 江苏省苏州市 移动 * GeoIP: Suzhou, Jiangsu, China * China Mobile
3710 186 2018-08-07 21:44:44 

NameError: name 'music_comments' is not defined

## 数据齐全
之后的操作就可以在这个新的 csv 数据里进行了，不想跑一边上面代码（不想在 ip 查询上虚耗生命的），只想用完整数据的，也只需从这一步开始，并读取 `Sina_Finance_Comments_All.csv` 文件即可。 

每一行代表一条评论，每一列代表每一条评论里的某一维度数据。很多列最后也没用上，但最开始并不知道，所以先都读取进来，不做筛选；一步步深入后才知道哪些有用、哪些无用。

In [161]:
#sinanews_comments.to_csv('Sina_Finance_Comments_All.csv', encoding='utf-8', line_terminator='\r\n')
df = pd.read_csv('Sina_Finance_Comments_All_20180811.csv',encoding='utf-8')
df.head()

,Unnamed: 0,No,page,nick,time,content,area,ip,ip_loc,length,against,agree,channel,hot,level,login_type,media_type,mid
0,0,1,1,潘昱材,2018-08-09 09:57:59,我手机三年没换 星巴克没去一次 我的房呢？,广东深圳,119.123.131.179,"广东省深圳市 电信 * GeoIP: Shenzhen, Guangdong, China ...",21,1533779879,0,cj,0,0,0,0,5B6B9FA6-777B83B3-68B55EBD-8C5-8E4
1,1,2,1,Qing诗卿,2018-08-09 09:54:57,回复 @amie是佳妹儿:存钱的重要性……买的基金涨涨跌跌已经不指望了,云南昆明,39.128.26.192,"云南省昆明市 移动 * GeoIP: Kunming, Yunnan, China * Ch...",35,1533779697,0,cj,0,3,0,0,5B6B9EEE-27801AC0-4DBD3FC8-8C5-8F6
2,2,3,1,Dcpzzzzzzzz,2018-08-09 09:49:12,哇，真的是树下野狐,浙江绍兴,124.160.212.121,"浙江省杭州市 联通 * GeoIP: Jiaxing, Zhejiang, China * ...",9,1533779352,0,cj,0,0,0,0,5B6B9D95-7CA0D479-AEA0727B-8C5-8C2
3,3,4,1,用户5650457955,2018-08-09 09:48:43,贷款买的房子，几个贷款买苹果消费的，少的可怜好吧，哪里来的砖家！？,湖北武汉,119.79.121.158,"湖北省武汉市 鹏博士 * GeoIP: Beijing, China * Computer ...",33,1533779323,0,cj,0,0,0,0,5B6B9D7B-774F799E-150CB2563-8C5-8A1
4,4,5,1,不喜欢了ky,2018-08-09 09:47:55,90后厉害了，你说他们，他们怼回去。[赞]。哈哈。,广东广州,113.115.59.2,"广东省 电信 * GeoIP: Dongguan, Guangdong, China * C...",25,1533779275,0,cj,0,0,0,0,5B6B9D4A-71733B02-5783407D-8C5-91A


In [37]:
df.describe()

,Unnamed: 0,No,page,length,against,agree,hot,level,login_type,media_type
count,3743.000000,3743.000000,3743.000000,3743.000000,3.743000e+03,3743.000000,3743.0,3743.000000,3743.000000,3743.0
mean,1871.000000,1872.000000,94.075341,23.344643,1.533702e+09,3.276249,0.0,0.378573,0.003473,0.0
std,1080.655357,1080.655357,54.032590,19.890127,2.824684e+04,56.614495,0.0,1.142278,0.074833,0.0
min,0.000000,1.000000,1.000000,1.000000,1.533645e+09,0.000000,0.0,0.000000,0.000000,0.0
25%,935.500000,936.500000,47.000000,10.000000,1.533691e+09,0.000000,0.0,0.000000,0.000000,0.0
50%,1871.000000,1872.000000,94.000000,18.000000,1.533697e+09,0.000000,0.0,0.000000,0.000000,0.0
75%,2806.500000,2807.500000,141.000000,30.000000,1.533709e+09,0.000000,0.0,0.000000,0.000000,0.0
max,3742.000000,3743.000000,188.000000,149.000000,1.533858e+09,2292.000000,0.0,19.000000,2.000000,0.0
